In [ ]:
import pandas as pd
import seaborn as sns
from rapidfuzz import process, fuzz
import matplotlib.pyplot as plt
import re

In [ ]:
# Step1: Extract only the street name
# Extract thing from we first see digit, until the first ,
# So everything before and after street name should not be there
def extract_street_name(job):
    if pd.isna(job):
        return None
    # Use regex to remove everything before the first digit and stop at the first comma
    match = re.search(r'\d.*?(?=,|$)', job)
    return match.group(0).strip() if match else job.strip()

In [ ]:
# Additional steps for street
# Step 1: Standardize street names in both sheet using a dictionary of replacements
def standardize_street_names_regex(name):
    if pd.isna(name):
        return None
    replacements = {
        r"\bDr\b\.?": "Drive",
        r"\bLn\b\.?": "Lane",
        r"\bSt\b\.?": "Street",
        r"\bAve\b\.?": "Avenue",
        r"\bBlvd\b\.?": "Boulevard",
        r"\bCt\b\.?": "Court",
        r"\bRd\b\.?": "Road",
        r"\bPl\b\.?": "Place",
    }
    for short, full in replacements.items():
        name = pd.Series([name]).str.replace(short, full, regex=True).iloc[0]
    return name.lower().strip()

In [ ]:
# Path to the Salesforce Excel file
salesforce_path = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\raw_data\All Move Ins Jan 2025.xlsx"

# Reading the Excel file into a DataFrame
salesforce = pd.read_excel(salesforce_path)
salesforce

In [ ]:
salesforce['Street Name'] = salesforce['Address'].apply(extract_street_name)

In [ ]:
salesforce

In [ ]:
salesforce['Standardized Street Name'] = salesforce['Street Name'].apply(standardize_street_names_regex)

In [ ]:
salesforce

In [ ]:
salesforce.describe()

## FOR ATL

#### Use second way to check ATL:

In [ ]:
# Load the data
# 11,186
purchase_orders_path_atl_all = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\raw_data\PurchaseOrders_Jan_2025.xlsx"
purchase_orders_atl_all = pd.read_excel(purchase_orders_path_atl_all, skiprows=1)

In [ ]:
purchase_orders_atl_all

In [ ]:
purchase_orders_atl_all.describe()

In [ ]:
purchase_orders_atl_all = purchase_orders_atl_all.dropna(subset=['Cost'])

In [ ]:
purchase_orders_atl_all['Street Name'] = purchase_orders_atl_all['Job'].apply(extract_street_name)

In [ ]:
# This for debug only to check if the street name is mapped correctly
print(purchase_orders_atl_all[['Job', 'Street Name']].drop_duplicates())

In [ ]:
purchase_orders_atl_all

In [ ]:
purchase_orders_atl_all['Street Name'].nunique()

In [ ]:
purchase_orders_atl_all[purchase_orders_atl_all['Cost'].isna()]

In [ ]:
purchase_orders_atl_all = purchase_orders_atl_all.dropna(subset=['Title'])

In [ ]:
purchase_orders_atl_all[purchase_orders_atl_all['Title'].isna()]

In [ ]:
purchase_orders_atl_all[purchase_orders_atl_all['Street Name'].isna()]

In [ ]:
purchase_orders_atl_all[purchase_orders_atl_all['Job'].isna()]

In [ ]:
# purchase_orders_atl_all[purchase_orders_atl_all['Street Name'] == "1104 Sallete Ct"].sort_values(by='Created Date')

In [ ]:
# Check the first 10 unique Job name 
print("Unique Job Names:", purchase_orders_atl_all['Job'].nunique())
print(purchase_orders_atl_all['Job'].unique()[:10])  # Print first 10 unique job names

In [ ]:
# Normalize the Job name for better looping
purchase_orders_atl_all['Job'] = purchase_orders_atl_all['Job'].str.strip().str.lower()

In [ ]:
# Step 2: Initialize a DataFrame to store results
final_results_atl = pd.DataFrame(columns=purchase_orders_atl_all.columns)

# Step 3: Loop through unique Job names
for job_name in purchase_orders_atl_all['Job'].unique():
    # Filter rows for the current job name
    job_rows = purchase_orders_atl_all[purchase_orders_atl_all['Job'] == job_name]
    
    # Sort rows by "Created Date" from earliest to latest
    job_rows = job_rows.sort_values(by='Created Date')

    # Search for "touch" in the "Title" column (case-insensitive)
    touch_rows = job_rows[job_rows['Title'].str.contains(r'touch', case=False, na=False)]
    
    if not touch_rows.empty:
        # Get the date of the first "touch" match
        first_touch_date = touch_rows.iloc[0]['Created Date']
        
        # Return all rows below the "touch" row (Created Date later)
        later_rows = job_rows[
            (job_rows['Created Date'] > first_touch_date) &  # Strictly later
            (~job_rows.index.isin(touch_rows.index))  # Exclude the actual "touch" row
            ].copy()
        
        if not later_rows.empty:
            # Separate positive and negative costs
            positive_rows = later_rows[later_rows['Cost'] > 0].copy()
            negative_rows = later_rows[later_rows['Cost'] < 0].copy()
            
            # Create a count dictionary for the absolute values of negative costs
            from collections import Counter
            negative_counts = Counter(negative_rows['Cost'].abs().tolist())
            
            # Iterate through each positive row and adjust if a match is found
            adjusted_positive_rows = []
            for _, pos_row in positive_rows.iterrows():
                pos_cost = pos_row['Cost']
                # If there's a matching negative cost, set this positive cost to 0
                if negative_counts[pos_cost] > 0:
                    negative_counts[pos_cost] -= 1
                    pos_row['Cost'] = 0  # set cost to zero instead of removing the row
                # Add the (possibly adjusted) positive row to the final list
                adjusted_positive_rows.append(pos_row)
            
            # Convert the adjusted positives back to a DataFrame
            adjusted_positive_df = pd.DataFrame(adjusted_positive_rows, columns=positive_rows.columns)
            
            # Since we are not including negative rows in the final results,
            # we only append the adjusted positive rows.
            final_results_atl = pd.concat([final_results_atl, adjusted_positive_df], ignore_index=True)

# final_results_atl now contains the rows after offsetting negative costs by zeroing out the matching positive costs.

In [ ]:
final_results_atl

In [ ]:
# Filter rows where 'Title' contains 'touch'
final_results_atl[final_results_atl['Title'].str.contains(r'touch', case=False, na=False)]

In [ ]:
final_results_atl.describe()

In [ ]:
# final_results_atl[final_results_atl['Street Name'] == "1104 Sallete Ct"]

In [ ]:
final_results_atl.shape # There are 1303 WOs

In [ ]:
final_results_atl['Standardized Street Name'] = final_results_atl['Street Name'].apply(standardize_street_names_regex)

In [ ]:
# Step 3: Format Created Date to MM/YYYY
final_results_atl['Created Date'] = pd.to_datetime(final_results_atl['Created Date'], errors='coerce')
final_results_atl['Month/Year'] = final_results_atl['Created Date'].dt.strftime('%m/%Y')

In [ ]:
# Step 4: Perform the merge with standardized street names
merged_data_standardized_atl = final_results_atl.merge(
    salesforce[['Standardized Street Name', 'Cleaned Name', 'Area Picklist']],
    left_on='Standardized Street Name',
    right_on='Standardized Street Name',
    how='inner'
)

In [ ]:
# merged_data_standardized_atl[merged_data_standardized_atl['Street Name'] == "1104 Sallete Ct"]

In [ ]:
merged_data_standardized_atl= merged_data_standardized_atl[merged_data_standardized_atl['Area Picklist'] == 'Atlanta']

In [ ]:
merged_data_standardized_atl['Cleaned Name'].unique()

In [ ]:
names_ATL = ['Clifford Senter',  'Jason Bishop', 'Jimmy Knox', 'Kirsten Davis', 'Nicholas Beuoy', 'Nicole Quiles', 'Paris Paggett', 'Ryan Worrell', 'Shaamar Moore', 'Trevor Stevens']
merged_data_standardized_atl = merged_data_standardized_atl[merged_data_standardized_atl['Cleaned Name'].isin(names_ATL)]

In [ ]:
merged_data_standardized_atl.describe()

In [ ]:
# Export the data for further use
export_columns = [
    "Title",
    "Street Name", 
    "Standardized Street Name", 
    "Created Date", 
    "Month/Year", 
    "Cleaned Name", 
    "Area Picklist", 
    "Cost"
]
export_data_atl = merged_data_standardized_atl[export_columns]

In [ ]:
export_data_atl

In [ ]:
df_ATL_agg = export_data_atl

# Calculate the average cost for each person per month
df_ATL_avg = df_ATL_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].mean()

# Pivot the data for plotting
df_ATL_pivot = df_ATL_avg.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_ATL_pivot.index = pd.to_datetime(df_ATL_pivot.index, format='%m/%Y')
df_ATL_pivot = df_ATL_pivot.sort_index()

# Calculate the overall average cost for ATL
overall_ATL_avg_cost = df_ATL_agg['Cost'].mean()

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_ATL_pivot.columns:
    plt.plot(df_ATL_pivot.index, df_ATL_pivot[column], marker='o', label=column)
    
# Add a horizontal dashed line for the overall average cost
plt.axhline(y=overall_ATL_avg_cost, color='grey', linestyle='--', linewidth=1, label=f'Overall Avg Cost (${overall_ATL_avg_cost:.2f})')

plt.title('ATL Average Monthly Move in Issues Cost per Field Super', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Project Manager', fontsize=10, loc='upper right', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Ensure 'Month/Year' column is in datetime format
export_data_atl['Month/Year'] = pd.to_datetime(export_data_atl['Month/Year'], format='%m/%Y', errors='coerce')

export_data_atl_Jan = export_data_atl[export_data_atl['Month/Year'] == '2025-01']
export_data_atl_Jan 

In [ ]:
df_ATL_agg = export_data_atl_Jan

# Calculate the average cost for each person per month
df_ATL_avg = df_ATL_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].mean()

# Pivot the data for plotting
df_ATL_pivot = df_ATL_avg.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_ATL_pivot.index = pd.to_datetime(df_ATL_pivot.index, format='%m/%Y')
df_ATL_pivot = df_ATL_pivot.sort_index()

# Calculate the overall average cost for ATL
overall_ATL_avg_cost = df_ATL_agg['Cost'].mean()

In [ ]:
overall_ATL_avg_cost

In [ ]:
df_ATL_avg 

In [ ]:
path = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\archive\WO_Cost_Jan2025_2_25_atl.xlsx"
export_data_atl_Jan.to_excel(path, index=False)

## FOR TX:

In [ ]:
# Load the Data
purchase_orders_path_tx_all = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\raw_data\PurchaseOrders_tx_Jan_2025.xlsx"

# Read the data from the new files
purchase_orders_tx_all = pd.read_excel(purchase_orders_path_tx_all, skiprows=1)

In [ ]:
purchase_orders_tx_all

In [ ]:
purchase_orders_tx_all['Street Name'] = purchase_orders_tx_all['Job'].apply(extract_street_name)

In [ ]:
purchase_orders_tx_all['Street Name'].nunique()

In [ ]:
# Normalize the Job name for better looping
purchase_orders_atl_all['Job'] = purchase_orders_atl_all['Job'].str.strip().str.lower()

In [ ]:
# Step 2: Initialize a DataFrame to store results
# Updated the loop to not only for each Job, show rows created after first see touch
# but also remove all negatives and also set positive value with same amount of negative cost found to zero
final_results_tx = pd.DataFrame(columns=purchase_orders_tx_all.columns)

# Step 3: Loop through unique Job names
for job_name_tx in purchase_orders_tx_all['Job'].unique():
    # Filter rows for the current job name
    job_rows_tx = purchase_orders_tx_all[purchase_orders_tx_all['Job'] == job_name_tx]
    
    # Sort rows by "Created Date" from earliest to latest
    job_rows_tx = job_rows_tx.sort_values(by='Created Date')
    
    # Search for "touch" in the "Title" column (case-insensitive)
    touch_rows_tx = job_rows_tx[job_rows_tx['Title'].str.contains(r'touch', case=False, na=False)]
    
    if not touch_rows_tx.empty:
        # Get the date of the first "touch" match
        first_touch_date_tx = touch_rows_tx.iloc[0]['Created Date']
        
        # Return all rows below the "touch" row (Created Date later)
        # Return all rows below the "touch" row (Created Date later)
        later_rows_tx = job_rows_tx[
            (job_rows_tx['Created Date'] > first_touch_date_tx) &  # Strictly later
            (~job_rows_tx.index.isin(touch_rows_tx.index))  # Exclude the actual "touch" row
            ].copy()
        
        if not later_rows_tx.empty:
            # Separate into positive and negative rows
            positive_rows = later_rows_tx[later_rows_tx['Cost'] > 0].copy()
            negative_rows = later_rows_tx[later_rows_tx['Cost'] < 0].copy()
            
            # Get a count of the absolute values of negative costs
            from collections import Counter
            negative_counts = Counter(negative_rows['Cost'].abs().tolist())

            # Keep track of final positive rows after adjustment
            adjusted_positive_rows = []
            
            # Iterate through each positive row to see if it can be offset by a negative
            for _, pos_row in positive_rows.iterrows():
                pos_cost = pos_row['Cost']
                # If there's a matching negative cost available, set the positive cost to 0
                if negative_counts[pos_cost] > 0:
                    negative_counts[pos_cost] -= 1
                    pos_row['Cost'] = 0
                # Add the (possibly adjusted) positive row to the final list
                adjusted_positive_rows.append(pos_row)
            
            # Convert the adjusted positives back to a DataFrame
            adjusted_positive_df = pd.DataFrame(adjusted_positive_rows, columns=positive_rows.columns)
            
            # Append these rows to the final results
            final_results_tx = pd.concat([final_results_tx, adjusted_positive_df], ignore_index=True)


In [ ]:
# Filter rows where 'Title' contains 'touch'
final_results_tx[final_results_tx['Title'].str.contains(r'touch', case=False, na=False)]

In [ ]:
final_results_tx['Standardized Street Name'] = final_results_tx['Street Name'].apply(standardize_street_names_regex)
final_results_tx.describe()

In [ ]:
final_results_tx.shape

In [ ]:
# Step 3: Format Created Date to MM/YYYY
final_results_tx['Created Date'] = pd.to_datetime(final_results_tx['Created Date'], errors='coerce')
final_results_tx['Month/Year'] = final_results_tx['Created Date'].dt.strftime('%m/%Y')

In [ ]:
final_results_tx.shape

In [ ]:
# Step 4: Perform the merge with standardized street names
merged_data_standardized_tx = final_results_tx.merge(
    salesforce[['Standardized Street Name', 'Cleaned Name', 'Area Picklist']],
    left_on='Standardized Street Name',
    right_on='Standardized Street Name',
    how='inner'
)

In [ ]:
merged_data_standardized_tx.describe()

In [ ]:
merged_data_standardized_tx[merged_data_standardized_tx['Cost'] < 0]

#### For DFW

In [ ]:
merged_data_standardized_dfw= merged_data_standardized_tx[merged_data_standardized_tx['Area Picklist'] == 'DFW']
merged_data_standardized_dfw['Cleaned Name'].unique()

In [ ]:
merged_data_standardized_dfw= merged_data_standardized_tx[merged_data_standardized_tx['Area Picklist'] == 'DFW']
names_DFW = ['Chase Wilson',  'Christopher Poujol','Christopher Silbaugh', 'Gilbert Sifuentes', 'Ricardo Martinez', 'Michael Woodson', 'Oscar Flores', 'William Goodson', 'William MacQueenette', 'Damon Nash']
merged_data_standardized_dfw = merged_data_standardized_dfw[merged_data_standardized_dfw['Cleaned Name'].isin(names_DFW)]

In [ ]:
merged_data_standardized_dfw['Cleaned Name'].unique()

In [ ]:
num_missing_cleaned_name_dfw = merged_data_standardized_dfw['Cleaned Name'].isna().sum()
num_missing_cleaned_name_dfw

In [ ]:
# Export the data for further use
export_columns = [
    "Title",
    "Street Name", 
    "Standardized Street Name", 
    "Created Date", 
    "Month/Year", 
    "Cleaned Name", 
    "Area Picklist", 
    "Cost"
]
export_data_dfw = merged_data_standardized_dfw[export_columns]
# # Define the full path to the 'Data' folder
# export_path_tx = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\TX_PurchaseOrders_AfterStandardizedMerge_Ordered.csv"

# # Export the data to the specified folder
# export_data.to_csv(export_path_tx, index=False)

In [ ]:
# df_DFW.to_csv("df_DFW.csv")

In [ ]:
# Group by WO_Number, Month/Year, and Cleaned Name, and sum the costs for duplicate work orders
# df_DFW_agg = df_DFW.groupby(['WO_Number', 'Month/Year', 'Cleaned Name'], as_index=False)['Cost'].sum()
# df_ATL_agg.to_csv("df_ATL_agg.csv")

df_DFW_agg = export_data_dfw

# Calculate the average cost for each person per month
df_DFW_avg = df_DFW_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].mean()

# Pivot the data for plotting
df_DFW_pivot = df_DFW_avg.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_DFW_pivot.index = pd.to_datetime(df_DFW_pivot.index, format='%m/%Y')
df_DFW_pivot = df_DFW_pivot.sort_index()

# Calculate the overall average cost for DFW
overall_DFW_avg_cost = df_DFW_agg['Cost'].mean()

In [ ]:
df_DFW_avg[df_DFW_avg['Cost'] < 0]

In [ ]:
export_data_dfw[(export_data_dfw['Month/Year'] == '08/2024') & (export_data_dfw['Cleaned Name'] == 'William Goodson')]

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_DFW_pivot.columns:
    plt.plot(df_DFW_pivot.index, df_DFW_pivot[column], marker='o', label=column)

# Add a horizontal dashed line for the overall average cost
plt.axhline(y=overall_DFW_avg_cost, color='grey', linestyle='--', linewidth=1, label=f'Overall Avg Cost (${overall_DFW_avg_cost:.2f})')

plt.title('DFW Average Monthly Move in Issues Cost per Field Super', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Project Manager', fontsize=10, loc='upper right', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plotting
# Make the plot to same range
plt.figure(figsize=(14, 8))
for column in df_DFW_pivot.columns:
    plt.plot(df_DFW_pivot.index, df_DFW_pivot[column], marker='o', label=column)

plt.title('DFW Average Cost Trend', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.ylim(0, 2000)  # Set the y-axis range from 0 to 2000
plt.legend(title='Cleaned Name', fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Ensure 'Month/Year' column is in datetime format
export_data_dfw['Month/Year'] = pd.to_datetime(export_data_dfw['Month/Year'], format='%m/%Y', errors='coerce')

export_data_dfw_Jan = export_data_dfw[export_data_dfw['Month/Year'] == '2025-01']
export_data_dfw_Jan 

In [ ]:
# path = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\archive\WO_Cost_Jan2025_2_25_dfw.xlsx"
# export_data_dfw_Jan.to_excel(path, index=False)

#### For HOU

In [ ]:
merged_data_standardized_hou= merged_data_standardized_tx[merged_data_standardized_tx['Area Picklist'] == 'Houston']
merged_data_standardized_hou['Cleaned Name'].unique()

In [ ]:
merged_data_standardized_hou= merged_data_standardized_tx[merged_data_standardized_tx['Area Picklist'] == 'Houston']
names_HOU = ['Angel Rosas', 'Tony Chavez', 'Bryant Johnson', 'Bryce Porter', 'Kenin Vargas', 'Kenneth Lee', 'Steve Wentz']
merged_data_standardized_hou = merged_data_standardized_hou[merged_data_standardized_hou['Cleaned Name'].isin(names_HOU)]

In [ ]:
merged_data_standardized_hou['Cleaned Name'].unique()

In [ ]:
num_missing_cleaned_name_hou = merged_data_standardized_hou['Cleaned Name'].isna().sum()
num_missing_cleaned_name_hou

In [ ]:
export_data_hou = merged_data_standardized_hou[export_columns]

In [ ]:
df_HOU_agg = export_data_hou

# Calculate the average cost for each person per month
df_HOU_avg = df_HOU_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].mean()

# Pivot the data for plotting
df_HOU_pivot = df_HOU_avg.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_HOU_pivot.index = pd.to_datetime(df_HOU_pivot.index, format='%m/%Y')
df_HOU_pivot = df_HOU_pivot.sort_index()

# Calculate the overall average cost for HOU
overall_HOU_avg_cost = df_HOU_agg['Cost'].mean()

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_HOU_pivot.columns:
    plt.plot(df_HOU_pivot.index, df_HOU_pivot[column], marker='o', label=column)

# Add a horizontal dashed line for the overall average cost
plt.axhline(y=overall_HOU_avg_cost, color='grey', linestyle='--', linewidth=1, label=f'Overall Avg Cost (${overall_HOU_avg_cost:.2f})')

plt.title('HOU Average Monthly Move in Issues Cost per Field Super', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Project Manager', fontsize=10, loc='upper left', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Ensure 'Month/Year' column is in datetime format
export_data_hou['Month/Year'] = pd.to_datetime(export_data_hou['Month/Year'], format='%m/%Y', errors='coerce')

export_data_hou_Jan = export_data_hou[export_data_hou['Month/Year'] == '2025-01']
export_data_hou_Jan 

In [ ]:
# path = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Data\Feb_2025\archive\WO_Cost_Jan2025_2_25_hou.xlsx"
# export_data_hou_Jan.to_excel(path, index=False)

#### Data Being Used:

In [ ]:
export_data_atl_Jan

In [ ]:
export_data_atl['Cost'].mean()

In [ ]:
export_data_dfw_Jan

In [ ]:
export_data_dfw ['Cost'].mean()

In [ ]:
export_data_hou_Jan

In [ ]:
export_data_hou['Cost'].mean()

In [ ]:
appended_table = pd.concat([export_data_atl_Jan, export_data_dfw_Jan, export_data_hou_Jan], ignore_index=True)

In [ ]:
appended_table

In [ ]:
# Show overall graph
average_cost = (
    appended_table.groupby(['Month/Year', 'Area Picklist'])['Cost']
    .mean()
    .reset_index()
)

# Pivot the data to prepare for plotting
pivot_data = average_cost.pivot(index='Month/Year', columns='Area Picklist', values='Cost')

# Plotting the line graph
plt.figure(figsize=(10, 6))
for column in pivot_data.columns:
    plt.plot(pivot_data.index, pivot_data[column],  marker='o', label=column)

plt.title('Average Cost by Area Picklist Over Time')
plt.xlabel('Month/Year')
plt.ylabel('Average Cost')
plt.legend(title='Project Manager', fontsize=10, loc='upper right', frameon=True)
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
appended_table['Month/Year'] = pd.to_datetime(appended_table['Month/Year'],  format="%m/%Y")

In [ ]:
appended_table.dtypes

In [ ]:
appended_table[appended_table['Title'].str.contains(r'touch', case=False, na=False)] # Make sure touch row itself is removed from final table

In [ ]:
# combined_path = r"C:\Users\Yijia Wang\Desktop\Open-House-Analysis\Summary\monthly_move_in_issues\Feb_2024\WO_Cost_Jan2025.xlsx"
# appended_table.to_excel(combined_path, index=False)

In [ ]:
appended_table

### Let's try monthly sum for all

##### ATL

In [ ]:
# Calculate the sum cost for each person per month
df_ATL_sum = df_ATL_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].sum()

# Pivot the data for plotting
df_ATL_pivot_sum = df_ATL_sum.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_ATL_pivot_sum.index = pd.to_datetime(df_ATL_pivot_sum.index, format='%m/%Y')
df_ATL_pivot_sum = df_ATL_pivot_sum.sort_index()

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_ATL_pivot_sum.columns:
    plt.plot(df_ATL_pivot_sum.index, df_ATL_pivot_sum[column], marker='o', label=column)

plt.title('ATL Sum Cost Trend', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Sum Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Cleaned Name', fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

##### DFW

In [ ]:
# Calculate the sum cost for each person per month
df_DFW_sum = df_DFW_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].sum()

# Pivot the data for plotting
df_DFW_pivot_sum = df_DFW_sum.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_DFW_pivot_sum.index = pd.to_datetime(df_DFW_pivot_sum.index, format='%m/%Y')
df_DFW_pivot_sum = df_DFW_pivot_sum.sort_index()

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_DFW_pivot_sum.columns:
    plt.plot(df_DFW_pivot_sum.index, df_DFW_pivot_sum[column], marker='o', label=column)

plt.title('DFW Sum Cost Trend', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Sum Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Cleaned Name', fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

##### HOU

In [ ]:
# Calculate the sum cost for each person per month
df_HOU_sum = df_HOU_agg.groupby(['Month/Year', 'Cleaned Name'], as_index=False)['Cost'].sum()

# Pivot the data for plotting
df_HOU_pivot_sum = df_HOU_sum.pivot(index='Month/Year', columns='Cleaned Name', values='Cost')

df_HOU_pivot_sum.index = pd.to_datetime(df_HOU_pivot_sum.index, format='%m/%Y')
df_HOU_pivot_sum = df_HOU_pivot_sum.sort_index()

In [ ]:
# Plotting
plt.figure(figsize=(14, 8))
for column in df_HOU_pivot_sum.columns:
    plt.plot(df_HOU_pivot_sum.index, df_HOU_pivot_sum[column], marker='o', label=column)

plt.title('HOU Sum Cost Trend', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Sum Cost ($)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Cleaned Name', fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Show overall graph
sum_cost = (
    appended_table.groupby(['Month/Year', 'Area Picklist'])['Cost']
    .sum()
    .reset_index()
)

# Pivot the data to prepare for plotting
pivot_data_sum = sum_cost.pivot(index='Month/Year', columns='Area Picklist', values='Cost')

# Plotting the line graph
plt.figure(figsize=(10, 6))
for column in pivot_data_sum.columns:
    plt.plot(pivot_data_sum.index, pivot_data_sum[column],  marker='o', label=column)

plt.title('Sum Cost by Area Picklist Over Time')
plt.xlabel('Month/Year')
plt.ylabel('Average Cost')
plt.legend(title='Area Picklist')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

#### Let's try distribution of cost

In [ ]:
# Histogram of Cost for Three Cities
plt.figure(figsize=(15, 5))  # width=15, height=5
# 1. Histogram for ATL
plt.subplot(1, 3, 1)
sns.histplot(data=df_ATL_agg, x='Cost', kde=False, bins=30, color='blue')
plt.title('Atlanta Cost Distribution')
plt.xlabel('Cost')
plt.ylabel('Frequency')

# 2. Histogram for DFW
plt.subplot(1, 3, 2)
sns.histplot(data=df_DFW_agg, x='Cost', kde=False, bins=30, color='green')
plt.title('DFW Cost Distribution')
plt.xlabel('Cost')
plt.ylabel('Frequency')

# 3. Histogram for HOU
plt.subplot(1, 3, 3)
sns.histplot(data=df_HOU_agg, x='Cost', kde=False, bins=30, color='red')
plt.title('Houston Cost Distribution')
plt.xlabel('Cost')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot of Cost for three cities
plt.figure(figsize=(15, 5))
# Create a combined DataFrame with an identifier column for region if you want to show them in one plot.
# Otherwise, you can just do separate subplots as shown here.

# 1. Box Plot for ATL
plt.subplot(1, 3, 1)
sns.boxplot(y='Cost', data=df_ATL_agg, color='blue')
plt.title('Atlanta Cost Distribution (Box Plot)')
plt.ylabel('Cost')

# 2. Box Plot for DFW
plt.subplot(1, 3, 2)
sns.boxplot(y='Cost', data=df_DFW_agg, color='green')
plt.title('DFW Cost Distribution (Box Plot)')
plt.ylabel('Cost')

# 3. Box Plot for HOU
plt.subplot(1, 3, 3)
sns.boxplot(y='Cost', data=df_HOU_agg, color='red')
plt.title('Houston Cost Distribution (Box Plot)')
plt.ylabel('Cost')

plt.tight_layout()
plt.show()


In [ ]:
# KDE plot for three cities
plt.figure(figsize=(15, 5))

# 1. KDE for ATL
plt.subplot(1, 3, 1)
sns.kdeplot(data=df_ATL_agg, x='Cost', fill=True, color='blue')
plt.title('ATL Cost Density')
plt.xlabel('Cost')
plt.ylabel('Density')

# 2. KDE for DFW
plt.subplot(1, 3, 2)
sns.kdeplot(data=df_DFW_agg, x='Cost', fill=True, color='green')
plt.title('DFW Cost Density')
plt.xlabel('Cost')
plt.ylabel('Density')

# 3. KDE for HOU
plt.subplot(1, 3, 3)
sns.kdeplot(data=df_HOU_agg, x='Cost', fill=True, color='red')
plt.title('HOU Cost Density')
plt.xlabel('Cost')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze correlations between costs and other factors

# 1. Costs vs. Work Order Type
cost_vs_type = appended_table.groupby('Title')['Cost'].mean().sort_values(ascending=False)

# 2. Costs vs. Area
cost_vs_area =appended_table.groupby('Area Picklist')['Cost'].mean().sort_values(ascending=False)

# 3. Costs vs. Time of Year
# Extract the month from the 'Month/Year' column
appended_table['Month'] = pd.to_datetime(appended_table['Month/Year']).dt.month
cost_vs_month = appended_table.groupby('Month')['Cost'].mean()
# Analyze correlations between costs and other factors

# 1. Costs vs. Work Order Type
cost_vs_type = appended_table.groupby('Title')['Cost'].mean().sort_values(ascending=False)

# 2. Costs vs. Area
cost_vs_area = appended_table.groupby('Area Picklist')['Cost'].mean().sort_values(ascending=False)

# 3. Costs vs. Time of Year
# Extract the month from the 'Month/Year' column
appended_table['Month'] = pd.to_datetime(appended_table['Month/Year']).dt.month
cost_vs_month = appended_table.groupby('Month')['Cost'].mean()

cost_vs_area

In [ ]:
cost_vs_month